**Nico Hillison**

Spring 2021

CS 251: Data Analysis and Visualization

Project 6: Supervised learning

In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

## Task 3: Preprocess full spam email dataset 

Before you build a Naive Bayes spam email classifier, run the full spam email dataset through your preprocessing code.

Download and extract the full **Enron** emails (*zip file should be ~29MB large*). You should see a base `enron` folder, with `spam` and `ham` subfolders when you extract the zip file (these are the 2 classes).

Run the test code below to check everything over.

### 3a) Preprocess dataset

In [2]:
import email_preprocessor as epp

#### Test `count_words` and `find_top_words`

In [3]:
word_freq, num_emails = epp.count_words()

In [4]:
print(f'You found {num_emails} emails in the datset. You should have found 32625.')

You found 32625 emails in the datset. You should have found 32625.


In [5]:
top_words, top_counts = epp.find_top_words(word_freq)
print(f"Your top 5 words are\n{top_words[:5]}\nand they should be\n['the', 'to', 'and', 'of', 'a']")
print(f"The associated counts are\n{top_counts[:5]}\nand they should be\n[277459, 203659, 148873, 139578, 111796]")

['the', 'to', 'and', 'of', 'a', 'in', 'for', 'you', 'is', 'enron', 'this', 'on', 'i', 'that', 's', 'subject', 'with', 'be', 'your', 'we', 'as', 'from', 'have', 'it', 'will', 'ect', 'are', 'or', 'at', 'by', 'not', 'our', 'if', 'com', 'company', 'all', 'please', 'an', 'has', 'can', 'hou', 'was', 'me', 'any', 'would', 'e', 'new', 'its', 'no', 'more', 'am', 'my', 'but', 't', 'may', 're', 'information', 'do', 'time', 'which', 'said', 'gas', 'about', 'business', 'up', 'energy', 'been', 'one', 'they', 'out', 'us', 'get', 'here', 'http', 'he', 'pm', 'email', 'message', 'cc', 'their', 'know', 'price', 'there', 'these', 'also', 'now', 'vince', 'need', 'other', 'so', 'only', 'power', 'what', 'm', 'thanks', 'over', 'mail', 'some', 'like', 'd', 'market', 'corp', 'stock', 'year', 'should', 'into', 'deal', 'just', 'sent', 'who', 'could', 'o', 'www', 'trading', 'inc', 'p', 'day', 'were', 'than', 'report', 'j', 'had', 'make', 'million', 'when', 'see', 'group', 'after', 'money', 'c', 'week', 'statements

### 3b) Make train and test splits of the dataset

Here we divide the email features into a 80/20 train/test split (80% of data used to train the supervised learning model, 20% we withhold and use for testing / prediction).

In [6]:
features, y = epp.make_feature_vectors(top_words, num_emails)

lst_to_wrds:  ['subject', 'what', 'up', 'your', 'cam', 'babe', 'what', 'are', 'you', 'looking', 'for', 'if', 'your', 'looking', 'for', 'a', 'companion', 'for', 'friendship', 'love', 'a', 'date', 'or', 'just', 'good', 'ole', 'fashioned', 'then', 'try', 'our', 'brand', 'new', 'site', 'it', 'was', 'developed', 'and', 'created', 'to', 'help', 'anyone', 'find', 'what', 'they', 're', 'looking', 'for', 'a', 'quick', 'bio', 'form', 'and', 'you', 're', 'on', 'the', 'road', 'to', 'satisfaction', 'in', 'every', 'sense', 'of', 'the', 'word', 'no', 'matter', 'what', 'that', 'may', 'be', 'try', 'it', 'out', 'and', 'youll', 'be', 'amazed', 'have', 'a', 'terrific', 'time', 'this', 'evening', 'copy', 'and', 'pa', 'ste', 'the', 'add', 'ress', 'you', 'see', 'on', 'the', 'line', 'below', 'into', 'your', 'browser', 'to', 'come', 'to', 'the', 'site', 'http', 'www', 'meganbang', 'biz', 'bld', 'acc', 'no', 'more', 'plz', 'http', 'www', 'naturalgolden', 'com', 'retract', 'counterattack', 'aitken', 'step', 'pre

ValueError: could not broadcast input array from shape (140,) into shape (32625,200)

In [ ]:
np.random.seed(0)
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(features, y)

In [ ]:
print('Shapes for train/test splits:')
print(f'Train {x_train.shape}, classes {y_train.shape}')
print(f'Test {x_test.shape}, classes {y_test.shape}')
print('\nThey should be:\nTrain (26100, 200), classes (26100,)\nTest (6525, 200), classes (6525,)')

### 3c) Save data in binary format

It adds a lot of overhead to have to run through your raw email -> train/test feature split every time you wanted to work on your project! In this step, you will export the data in memory to disk in a binary format. That way, you can quickly load all the data back into memory (directly in ndarray format) whenever you want to work with it again. No need to parse from text files!

- Use numpy's `save` function to make six files in `.npy` format (e.g. `email_train_x.npy`, `email_train_y.npy`, `email_train_inds.npy`, `email_test_x.npy`, `email_test_y.npy`, `email_test_inds.npy`).

## Task 4: Naive Bayes Classifier

After finishing your email preprocessing pipeline, implement the one other supervised learning algorithm we we will use to classify email, **Naive Bayes**.

### 4a) Implement Naive Bayes

In `naive_bayes.py`, implement the following methods:
- Constructor
- `train(data, y)`: Train the Naive Bayes classifier so that it records the "statistics" of the training set: class priors (i.e. how likely an email is in the training set to be spam or ham?) and the class likelihoods (the probability of a word appearing in each class — spam or ham).
- `predict(data)`: Combine the class likelihoods and priors to compute the posterior distribution. The predicted class for a test sample is the class that yields the highest posterior probability.
- `accuracy(y, y_pred)`: The usual definition :)


#### Bayes rule ingredients: Priors and likelihood (`train`)

To compute class predictions (probability that a test example belong to either spam or ham classes), we need to evaluate **Bayes Rule**. This means computing the priors and likelihoods based on the training data.

**Prior:** $$P_c = \frac{N_c}{N}$$ where $P_c$ is the prior for class $c$ (spam or ham), $N_c$ is the number of training samples that belong to class $c$ and $N$ is the total number of training samples.

**Likelihood:** $$L_{c,w} = \frac{N_{c,w} + 1}{N_{c} + M}$$ where
- $L_{c,w}$ is the likelihood that word $w$ belongs to class $c$ (*i.e. what we are solving for*)
- $N_{c,w}$ is the total count of **word $w$** in emails that are only in class $c$ (*either spam or ham*)
- $N_{c}$ is the total number of **all words** that appear in emails of the class $c$ (*total number of words in all spam emails or total number of words in all ham emails*)
- $M$ is the number of features (*number of top words*).

#### Bayes rule ingredients: Posterior (`predict`)

To make predictions, we now combine the prior and likelihood to get the posterior:

**Log Posterior:** $$Log(\text{Post}_{i, c}) = Log(P_c) + \sum_{j \in J_i}x_{i,j}Log(L_{c,j})$$

 where
- $\text{Post}_{i,c}$ is the posterior for class $c$ for test sample $i$(*i.e. evidence that email $i$ is spam or ham*). We solve for its logarithm.
- $Log(P_c)$ is the logarithm of the prior for class $c$.
- $x_{i,j}$ is the number of times the jth word appears in the ith email.
- $Log(L_{c,j})$: is the log-likelihood of the jth word in class $c$.

In [ ]:
from naive_bayes_multinomial import NaiveBayes

#### Test `train`

In [ ]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.random(size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your class priors are: {nbc.class_priors}\nand should be          [0.24 0.26 0.25 0.25].')
print(f'Your class likelihoods shape is {nbc.class_likelihoods.shape} and should be (4, 6).')
print(f'Your likelihoods are:\n{nbc.class_likelihoods}')


test_likelihoods = np.array([[0.15116, 0.18497, 0.17571, 0.1463 , 0.16813, 0.17374],
       [0.16695, 0.17437, 0.15742, 0.16887, 0.15677, 0.17562],
       [0.14116, 0.1562 , 0.19651, 0.17046, 0.17951, 0.15617],
        [0.18677, 0.18231, 0.15884, 0.12265, 0.16755, 0.18187]])
print(f'and should be\n{test_likelihoods}')

#### Test `predict`

In [ ]:
num_test_classes = 4
np.random.seed(0)
data_train = np.random.randint(low=0, high=num_test_classes, size=(100, 10))
data_test = np.random.randint(low=0, high=num_test_classes, size=(15, 10))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_train, y_test)
test_y_pred = nbc.predict(data_test)

print(f'Your predicted classes are\n{test_y_pred}\nand should be\n[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]')

### 4b) Spam filtering

Let's start classifying spam email using the Naive Bayes classifier.

- Use `np.load` to load in the train/test split that you created last week.
- Use your Naive Bayes classifier on the Enron email dataset!

**Question 7:** Print out the accuracy that you get on the test set with Naive Bayes. It should be roughly 89%.

In [ ]:
import email_preprocessor as ep

In [ ]:
x_train = np.load('data/enron_preprocessed/email_train_x.npy')
y_train = np.load('data/enron_preprocessed/email_train_y.npy')
inds_train = np.load('data/enron_preprocessed/email_train_inds.npy')
x_test = np.load('data/enron_preprocessed/email_test_x.npy')
y_test = np.load('data/enron_preprocessed/email_test_y.npy')
inds_test = np.load('data/enron_preprocessed/email_test_inds.npy')

In [ ]:
nbc = NaiveBayes(num_classes=2)
nbc.train(x_train, y_train)
y_pred = nbc.predict(x_test)
print(nbc.accuracy(y_test, y_pred))

### 4c) Confusion matrix

To get a better sense of the errors that the Naive Bayes classifer makes, you will create a confusion matrix. 

- Implement `confusion_matrix` in `naive_bayes.py`.
- Print out a confusion matrix of the spam classification results.

**Debugging guidelines**:
1. The sum of all numbers in your 2x2 confusion matrix should equal the number of test samples (6525).
2. The sum of your spam row should equal the number of spam samples in the test set (3193)
3. The sum of your ham row should equal the number of spam samples in the test set (3332)

In [ ]:
nbc = NaiveBayes(num_classes=2)
nbc.train(x_train, y_train)
y_pred = nbc.predict(x_test)
print(nbc.confusion_matrix(y_test, y_pred))


**Question 8:** Interpret the confusion matrix, using the convention that positive detection means spam (*e.g. a false positive means classifying a ham email as spam*). What types of errors are made more frequently by the classifier? What does this mean (*i.e. X (spam/ham) is more likely to be classified than Y (spam/ham) than the other way around*)?

**Reminder:** Look back and make sure you are clear on which class indices correspond to spam/ham.

**Answer 8:** there are 150 false posities and 552 false negatives, basically saying that false positive means that 702 emails were misclassified (150 misclassified as spam, 552 as ham). The classifier did not do a good job avoiding the spam emails and placed them into ham.

### 4d) Investigate the misclassification errors

Numbers are nice, but they may not the best for developing your intuition. Sometimes, you want to see what an misclassification *actually looks like* to help you improve your algorithm. Here, you will take a false positive and a false negative misclassification and retrieve the actual text of the email so see which emails produced the error.

- Determine the index of the **FIRST** false positive and false negative misclassification — i.e. 2 indices in total. Remember to use your `test_inds` array to look up the index of the emails BEFORE shuffling happened.
- Implement the function `retrieve_emails` in `email_preprocessor.py` to return the string of the raw email at the error indices.
- Call your function to print out the two emails that produced misclassifications.

**Question 9:** Does it seem reasonable that each email message was misclassified? Why?

**Answer 9:** No, because we know that the accuracy of the classifier is at least 90% so there is no way all emails were misclassified. 

In [8]:
print('first false positive: ', epp.retrieve_emails([3]))
print('first false negative: ', epp.retrieve_emails([44]))

first false positive:  ['Subject: big range of all types of downloadable software . need software ? click here . our american professors like their literature clear , cold , pure and very dead . being another character is more interesting than being yourself .']
first false negative:  ['Subject: herbal viagra 30 day trial . . . oncxbv exit list instructions pmoney']


## Task 5: Comparison with KNN


- Run a similar analysis to what you did with Naive Bayes above. When computing accuracy on the test set, you may want to reduce the size of the test set (e.g. to the first 500 emails in the test set).
- Copy-paste your `confusion_matrix` method into `knn.py` so that you can run the same analysis on a KNN classifier.

In [ ]:
from knn import KNN

In [ ]:
knn = KNN(num_classes=2)
knn.train(x_test, y_test)
y_pred = knn.predict(x_test, 500)
print(knn.accuracy(y_test, y_pred))

**Question 10:** What accuracy did you get on the test set (potentially reduced in size)?

**Question 11:** How does the confusion matrix compare to that obtained by Naive Bayes (*If you reduced the test set size, keep that in mind*)?

**Question 12:** Briefly describe at least one pro/con of KNN compared to Naive Bayes on this dataset.

**Question 13:** When potentially reducing the size of the test set here, why is it important that we shuffled our train and test set?

**Answer 10:** 76% accuracy

**Answer 11:** As a first glance, it is still a big difference, by 14% so I believe the Naive Bayes one is more accurate either way. 

**Answer 12:** KNN is a lot slower (con), KNN is non-paremetic (pro)

**Answer 13:** so that when we are comparing both, because of order the KNN does not lead to worse accuracy, though it did

## Extensions

### 0. Classify your own datasets

- Find datasets that you find interesting and run classification on them using your KNN algorithm (and if applicable, Naive Bayes). Analysis the performance of your classifer.

### 1. Better text preprocessing

- If you look at the top words extracted from the email dataset, many of them are common "stop words" (e.g. a, the, to, etc.) that do not carry much meaning when it comes to differentiating between spam vs. non-spam email. Improve your preprocessing pipeline by building your top words without stop words. Analyze performance differences.

### 2. Feature size

- Explore how the number of selected features for the email dataset influences accuracy and runtime performance.

### 3. Distance metrics
- Compare KNN performance with the $L^2$ and $L^1$ distance metrics

### 4. K-Fold Cross-Validation

- Research this technique and apply it to data and your KNN and/or Naive Bayes classifiers.

### 5. Email error analysis

- Dive deeper into the properties of the emails that were misclassified (FP and/or FN) by Naive Bayes or KNN. What is their word composition? How many words were skipped because they were not in the training set? What could plausibly account for the misclassifications?